#0. 実行の準備

1. 高速に計算できるようにGPUを有効にする<br>
    メニューバー「ランタイム」→「ランタイムのタイプを変更」→「T4」

1. 画面「ドライブにコピーを保存」をクリック<br>
    ファイル名が「DS入門4回目_Hirose.ipynbのコピー」に変更される

1. この演習で利用するモジュール(プログラムの部品)をインストールする。<br>
    次のコマンドを実行(ボタンをクリック)

In [ ]:
# グラフに日本語を表示するモジュールをインストール
!pip install japanize_matplotlib

# 株価のろうそく足チャートを表示するモジュールをインストール
!pip install mplfinance

#1. 気象データ weather.csv

気象庁のHPより過去のデータがダウンロードできる。
[気象庁のHP](https://www.data.jma.go.jp/risk/obsdl/index.php)

1900年から2025年4月までの、東京、札幌、福岡の月平均気温(templature)、月当たりの降水量(participation)、月当たりの日照時間(sunshine)として、weather.csvとしてGitHubに保存している。

これより、地球温暖化は本当か！？をテーマに、データ分析する。

## 1.1 データの読み込み

データを読み込み、保存された内容を確認する

PythonでURLを指定してCSV(カンマで区切られたファイル)ファイルを読み込む
```
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
df = pd.read_csv(url)
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])

display(df)


##1-2 データをグラフで表示

東京に絞って、1900年か2025年まで月平均気温がどのように変化したかグラフで表示したい

- 東京のデータだけ抽出したい
```
city_data = df[df['city'] == 'Tokyo']
```

- X軸を日付、Y軸を東京の月別平均気温(templature)として折れ線グラフ
```
plt.plot(city_data['date'], city_data['templature'], label='東京')
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])

# 東京の気温データだけを抽出
city_data = df[df['city'] == 'Tokyo']

# 折れ線グラフの描画
plt.figure(figsize=(12, 6))
#X軸を日付(date)、Y軸を月平均気温(templature)
plt.plot(city_data['date'], city_data['templature'], label='東京')
plt.title('東京の気温の推移')
plt.xlabel('日付')
plt.ylabel('気温（℃）')
plt.grid(True)
plt.legend()
plt.show()
plt.close()

##1-3 気温データを年ごとに平均化する

12か月のデータの平均値をとり、年ごとの平均気温データを作る
都市別に年ごとにグループ化(groupby)し、平均値(mean)を計算する
```
df_year = df.groupby(['city', 'year'])['templature'].mean().reset_index()
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])
# 都市別に年ごとにグループ化して平均気温を算出
df['year'] = df['date'].dt.year
df_year = df.groupby(['city', 'year'])['templature'].mean().reset_index()

display(df_year)

# 東京の気温データだけを抽出
city_data = df_year[df_year['city'] == 'Tokyo']

# 折れ線グラフの描画
plt.figure(figsize=(12, 6))
#X軸を日付(date)、Y軸を月平均気温(templature)
plt.plot(city_data['year'], city_data['templature'], label='東京')
plt.title('東京の気温の推移')
plt.xlabel('日付')
plt.ylabel('気温（℃）')
plt.grid(True)
plt.legend()
plt.show()
plt.close()

# 移動平均とは？

**移動平均（Moving Average, MA）** とは、時系列データの変動を平滑化し、傾向（トレンド）を把握するための基本的な手法。  
株価・気温・売上など、変動のあるデータに対してよく使われる。

---

## 単純移動平均（SMA: Simple Moving Average）

###  定義  
直近 n 期間のデータの単純な平均

###  数式

$$
SMA_t = \frac{1}{n} \sum_{i=0}^{n-1} x_{t-i}
$$

- $ SMA_t $：時刻 t における移動平均
- $ x_t $：時刻 t の元データ（例：株価）
- $ n $：平均を取る期間

---

##  加重移動平均（WMA: Weighted Moving Average）

###  定義  
各データに重みをかけて平均する方法。最近のデータに大きな重みを与える

###  数式

$$
WMA_t = \frac{\sum_{i=0}^{n-1} w_i \cdot x_{t-i}}{\sum_{i=0}^{n-1} w_i}
$$

- $ w_i $：各時点に対応する重み

---

##  指数移動平均（EMA: Exponential Moving Average）

###  定義  
過去すべてのデータを指数的に減衰させながら平均する方法

###  数式（再帰的）

$$
EMA_t = \alpha \cdot x_t + (1 - \alpha) \cdot EMA_{t-1}
$$

- $ \alpha $ ：平滑化係数（例：$ \alpha = \frac{2}{n+1} $）

---

## 用途別の選び方

| 種類 | 特徴 | 主な用途 |
|------|------|----------|
| SMA  | シンプルで計算が容易 | 基本的なトレンド分析 |
| WMA  | 最近のデータに重み | 素早いトレンド反応 |
| EMA  | すべての過去データを考慮 | テクニカル指標で一般的 |


##算術移動平均の実装

算術移動平均を実装するには、**rolling(window=期間数).mean()**
```
kikan = 7
tokyo_data['SMA'] = tokyo_data['templature'].rolling(window=kikan).mean()
```

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])
# 都市別に年ごとにグループ化して平均気温を算出
df['year'] = df['date'].dt.year
df_year = df.groupby(['city', 'year'])['templature'].mean().reset_index()
#display(df_year)

# 東京の気温データだけを抽出
city_data = df_year[df_year['city'] == 'Tokyo'].copy()

# 移動平均（kikanカ月）を計算
kikan = 7
city_data['SMA'] = city_data['templature'].rolling(window=kikan).mean()

# グラフ描画
plt.figure(figsize=(12, 6))
plt.plot(city_data['year'], city_data['templature'], label='実測値')
plt.plot(city_data['year'], city_data['SMA'], label=f'{kikan}期間移動平均')
plt.title('東京の気温と移動平均（7日）')
plt.xlabel('日付')
plt.ylabel('気温（℃）')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


###問題1
東京と札幌(sapporo)の年間平均気温を比較し、平均気温が上がっているか確かめる

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])
# 都市別に年ごとにグループ化して平均気温を算出
df['year'] = df['date'].dt.year
df_year = df.groupby(['city', 'year'])['templature'].mean().reset_index()
#display(df_year)

# 東京の気温データだけを抽出
city_data = df_year[df_year['city'] == 'Tokyo'].copy()

# 移動平均（kikanカ月）を計算
kikan = 7
city_data['SMA'] = city_data['templature'].rolling(window=kikan).mean()

# グラフ描画
plt.figure(figsize=(12, 6))
plt.plot(city_data['year'], city_data['templature'], label='実測値')
plt.plot(city_data['year'], city_data['SMA'], label=f'{kikan}年移動平均')
plt.title('東京の気温と移動平均（7日）')
plt.xlabel('日付')
plt.ylabel('気温（℃）')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


###問題２

東京の日照時間の推移をグラフで表示する。

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/weather.csv'
# CSVを読み込む
df = pd.read_csv(url)
# 日付をdatetime型に変換
df['date'] = pd.to_datetime(df['date'])
# 都市別に年ごとにグループ化して平均気温を算出
df['year'] = df['date'].dt.year
df_year = df.groupby(['city', 'year'])['templature'].mean().reset_index()
#display(df_year)

# 東京の気温データだけを抽出
city_data = df_year[df_year['city'] == 'Tokyo'].copy()

# 移動平均（kikanカ月）を計算
kikan = 7
city_data['SMA'] = city_data['sunshine'].rolling(window=kikan).mean()

# グラフ描画
plt.figure(figsize=(12, 6))
plt.plot(city_data['year'], city_data['templature'], label='東京実測値')
plt.plot(city_data['year'], city_data['SMA'], label=f'東京{kikan}年移動平均')
plt.title('東京の日照時間と移動平均（7年）')
plt.xlabel('日付')
plt.ylabel('気温（℃）')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

#株価データ

## 2－1 株価データとは
株価データを提供しているサイト[Investing.com](https://jp.investing.com/equities)

株価情報は、終値(おわりね)、始値(はじめね)、高値、安値、出来高(一日の取引数)として保存されている。

(株)inpexの2020年から2025年5月7日までの株価情報を取得し、GitHubに1605.csvとして保存されている。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/1605.csv'

# CSVを読み込む
df = pd.read_csv(url)

# 'date' を日付型に変換してインデックスに設定
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
#データの確認
display(df)


##2-2 株価の可視化

このデータを株価チャート(ろうそく足チャート)で表示するプログラム

[ろうそく足チャート](https://drive.google.com/file/d/1Hu2NkdC1827ZOZhYyw9Y_CFLEyRVcsy0/view?usp=sharing)



In [ ]:
# ろうそく足チャートモジュールのインポート
import pandas as pd
import mplfinance as mpf

# データの読み込み
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/1605.csv'
df = pd.read_csv(url)
# インデックスに日付を設定
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
#display(df)

# ローソク足チャートの描画
mpf.plot(df, type='candle', style='charles', volume=True,title='INPEX(1605)', ylabel='Price', ylabel_lower='Volume',figratio=(12, 6))

##2-3 表示の指定

現状は2020年からデータが多すぎてWarningが表示される。

そこで、2025年1月1日からのデータに限定してグラフを表示する。


In [ ]:
# ろうそく足チャートモジュールのインポート
import pandas as pd
import mplfinance as mpf

# データの読み込み
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/1605.csv'
df = pd.read_csv(url)
# インデックスに日付を設定
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

#日付を設定
df = df[df.index >= '2025-01-01']
display(df)

# ローソク足チャートの描画
mpf.plot(df, type='candle', style='charles', volume=True,title='INPEX(1605)', ylabel='Price', ylabel_lower='Volume',figratio=(12, 6))


##2-4 株価を予測

株価を予測したい。株価で一番重要視されているのは、終値である。
終値だけのチャートを表示するプログラム。




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# GitHubのraw CSVファイルURL
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/1605.csv'

# CSVを読み込む
df = pd.read_csv(url)

# 'date' を日付型に変換してインデックスに設定
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
#日付を設定
#df = df[df.index >= '2025-01-01']
display(df)

# 折れ線グラフの作成（終値）
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='Close Price')
plt.title('Stock Closing Price Over Time')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.legend()
plt.show()
plt.close()

##2-5 10日後の株価をGRUで予測する


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# データ読み込み（GitHub等から）
url = 'https://raw.githubusercontent.com/HirooHirose/ds_Hirose/main/1605.csv'
df = pd.read_csv(url)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df = df[df.index >= '2025-01-01'][['Close']]

# スケーリング
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df)

# 時系列データ作成
def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X, y = create_sequences(scaled, seq_length)

# GRUモデル構築
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(seq_length, 1)),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=100, verbose=0)

# 10日間を再帰的に予測
last_seq = scaled[-seq_length:].reshape(1, seq_length, 1)
preds = []
for _ in range(10):
    pred = model.predict(last_seq, verbose=0)[0, 0]
    preds.append(pred)
    last_seq = np.append(last_seq[:, 1:, :], [[[pred]]], axis=1)

# 逆正規化して表示
pred_prices = scaler.inverse_transform(np.array(preds).reshape(-1, 1))
future_dates = pd.date_range(df.index[-1] + pd.Timedelta(days=1), periods=10, freq='B')
forecast_df = pd.DataFrame(pred_prices.round(1), index=future_dates, columns=['Predicted Close'])

print(f"5月7日以降10日間の予測\n {forecast_df}")

## 結果をグラフで表示する

In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフ描画
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='Actual Close', color='blue')
plt.plot(forecast_df.index, forecast_df['Predicted Close'], label='Predicted Close (GRU)', color='red', linestyle='--', marker='o')
plt.axvline(x=df.index[-1], color='gray', linestyle='dashed', label='Prediction Start')
plt.title('INPEX (1605) 10days forcast price by GRU')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.grid(True)
plt.legend()
#plt.tight_layout()
plt.show()
plt.close()